## Day 23: Coprocessor Conflagration

http://adventofcode.com/2017/day/23

### Part 1

In [1]:
from collections import defaultdict

def run(commands):
    registers = defaultdict(int)
    mul_invocations = 0

    # Returns a number if it's a number or registers[x] otherwise
    def value(x):
        return registers[x] if x.isalpha() else int(x)

    command_number = 0
    
    while(0 <= command_number < len(commands)):
        fields = commands[command_number].split()
        command_name = fields[0]
        parameters = fields[1:]
        
        x, y = parameters[0], parameters[1]
        if command_name == 'set':
            registers[x] = value(y)
        elif command_name == 'sub':
            registers[x] -= value(y)
        elif command_name == 'mul':
            registers[x] *= value(y)
            mul_invocations += 1
        elif command_name == 'jnz':
            if value(x) != 0:
                command_number += value(y) - 1
                    
        command_number += 1
        
    return mul_invocations, registers['h'] # the latter added for Part 2

Right, I'll set this going and head to Sainsbury's. Wish me luck.

In [2]:
with open('input', 'r') as f:
    input_code = f.readlines()
    
%time answer = run(input_code)

CPU times: user 82.7 ms, sys: 210 µs, total: 82.9 ms
Wall time: 81.7 ms


<s>That's never going to stop. There's most likely a slow long loop somewhere, let's see what it is.</s> Reading the question properly helped here.

In [3]:
answer

(8281, 1)

### Part 2

I find it hard to work out what's going on with the quasi-assembler, so convert it to python and optimise, testing against the input for the first part each time. First add line numbers to the program and amend the `jnz` commands so they point to absolute rather than relative positions, which will make things easier to analyse.

In [4]:
def add_linenumbers_to(program):
    result = []
    
    for line_number, line in enumerate(program):
        fields = line.strip().split()
        
        if fields[0] == 'jnz':
            fields[2] = str(line_number + int(fields[2]))
            
        result.append(' '.join([str(line_number)] + fields))
        
    return result

debug_code = add_linenumbers_to(input_code)
print('\n'.join(debug_code))

0 set b 93
1 set c b
2 jnz a 4
3 jnz 1 8
4 mul b 100
5 sub b -100000
6 set c b
7 sub c -17000
8 set f 1
9 set d 2
10 set e 2
11 set g d
12 mul g e
13 sub g b
14 jnz g 16
15 set f 0
16 sub e -1
17 set g e
18 sub g b
19 jnz g 11
20 sub d -1
21 set g d
22 sub g b
23 jnz g 10
24 jnz f 26
25 sub h -1
26 set g b
27 sub g c
28 jnz g 30
29 jnz 1 32
30 sub b -17
31 jnz 1 8


Human compilation first pass.

In [5]:
a = 0
b = 93
c = b
h = 0

multiplications = 0

if a != 0: # The clause to make everything take ages
    b = b * 100 + 100000
    multiplications += 1
    c = b + 17000

while True: # 8 - 31
    f = 1
    d = 2

    while d != b: # 10 - 23
        e = 2

        while e != b: # 11 - 19
            g = d * e - b
            multiplications += 1

            if g == 0:
                f = 0

            e = e + 1

        d = d + 1

    if f == 0:
        h = h + 1

    g = b - c

    if g == 0:
        print(multiplications, h)
        break

    b = b + 17

8281 1


That works. Now optimise the inner loop. This increases `e` from 2 until `d * e == b`, when it sets `f` to `0`, which seems to be a flag that `d` is a factor of `b`. It then continues for `b - d*e` loops for no good reason.

Aha. Lines 10-23 are working out if `b` is prime and increasing `h` if it has a factor. 

Human compilation -O1.

In [6]:
import math

a = 0
b = 93
c = b
g = 0
h = 0

multiplications = 0

if a != 0: # The clause to make everything take ages
    b = b * 100 + 100000
    multiplications += 1
    c = b + 17000

while True: # 8 - 31
    for d in range(2, math.floor(math.sqrt(b))):
        if b % d == 0:
            h = h + 1
            break
    multiplications += (b-2) ** 2
    
    g = b - c

    if b == c:
        print(multiplications, h)
        break

    b = b + 17

8281 1


OK, let's see what happens when `a` is set to 1.

In [7]:
import math

a = 1
b = 93
c = b
g = 0
h = 0

multiplications = 0

if a != 0: # The clause to make everything take ages
    b = b * 100 + 100000
    multiplications += 1
    c = b + 17000

while True: # 8 - 31
    for d in range(2, math.floor(math.sqrt(b))):
        if b % d == 0:
            h = h + 1
            break
    multiplications += (b-2) ** 2
    
    g = b - c

    if b == c:
        print(multiplications, h)
        break

    b = b + 17

13914400804305 911


That's correct.

In [8]:
b = 9300 + 100000
c = b + 17000

sum(any(x % f == 0 for f in range(2, math.floor(math.sqrt(x)))) for x in range(b, c+1, 17))

911

I'm not sure that's any easier to read than the assembly language. 

I enjoyed that one. 